### Temperature and Precipitation

Scource: https://kilthub.cmu.edu/articles/dataset/Compiled_daily_temperature_and_precipitation_data_for_the_U_S_cities/7890488

Compiled historical daily temperature and precipitation data for selected 210 U.S. cities

Each file provides available historical daily maximum and minimum temperature and daily precipitation data for one U.S. city. File was named by the city's current active weather station ID (GHCN ID).

Each city may include records from one or multiple stations. Listed latitude and longitude for each city are from the city's current active weather station.



For each file:

1. read as csv, remove index column
2. drop null values
3. range selected: 2017 - 2021
4. aggregate by year then month and take monthly average

In [1]:
import re
import pandas as pd

In [2]:
!ls

EDA_images                city_temp_precip.ipynb    temp_city_stationID.csv
Untitled.ipynb            geo.ipynb                 temp_mon_avg.csv
W210_Capstone             good_temp.csv             zone-Copy1.ipynb
cb_2018_us_state_500k.zip ideation                  zone.ipynb
city_state_code.csv       state_ab.csv
city_temp_prec.csv        station_state.csv


In [3]:
city_info = pd.read_csv('./temp_prec/city_info.csv', index_col=0)
city_info.head()

,Name,ID,Lat,Lon,Stn.Name,Stn.stDate,Stn.edDate
1,Lander,USW00024021,42.8153,-108.7261,LANDER WBO,1892-01-01,1946-05-28
2,Lander,USW00024021,42.8153,-108.7261,LANDER HUNT FIELD,1946-05-29,2021-12-31
3,Cheyenne,USW00024018,41.1519,-104.8061,CHEYENNE WBO,1871-01-01,1935-08-31
4,Cheyenne,USW00024018,41.1519,-104.8061,CHEYENNE MUNICIPAL ARPT,1935-09-01,2021-12-31
5,Wausau,USW00014897,44.9258,-89.6256,Wausau Record Herald,1896-01-01,1941-12-31


In [26]:
import pandas as pd

city_info = pd.read_csv('./temp_prec/city_info.csv', index_col=0)
city_info = city_info.drop_duplicates(subset=['Name'])[['Name', 'ID']]# each city may have duplcite rows due to diff stations. 

city_id = city_info.ID.to_list()
result = pd.DataFrame()

for c_id in city_id:
    # read temp data of each city, cleanup
    df = pd.read_csv(f'./temp_prec/{c_id}.csv', index_col=0)
    df.dropna(inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df['year'] = df['Date'].dt.year
    df['month'] = df['Date'].dt.month
    
    # select year 2017-2021
    df = df[(df['year']>= 2017) & (df['year'] <= 2021)]
    # aggregate by year month and take average
    avg_df = df.groupby(['year', 'month']).mean().reset_index()
    
    # assign city name based on id
    avg_df['ID'] = c_id
    avg_df['name'] = city_info[city_info['ID']==c_id]['Name'].values[0]
    
    # concat all city temp data
    result = pd.concat([result, avg_df], axis=0)

result

,year,month,tmax,tmin,prcp,ID,name
0,2017,1,22.225806,2.258065,0.040968,USW00024021,Lander
1,2017,2,38.964286,17.142857,0.039643,USW00024021,Lander
2,2017,3,54.032258,29.548387,0.150000,USW00024021,Lander
3,2017,4,53.900000,30.966667,0.108667,USW00024021,Lander
4,2017,5,65.096774,39.677419,0.070968,USW00024021,Lander
...,...,...,...,...,...,...,...
55,2021,8,88.651613,72.729032,0.248387,USW00013876,Birmingham
56,2021,9,84.295833,65.304167,0.065000,USW00013876,Birmingham
57,2021,10,77.038710,58.654839,0.302903,USW00013876,Birmingham
58,2021,11,63.390000,41.126667,0.054333,USW00013876,Birmingham


In [28]:
# result.to_csv('city_temp_prec.csv')

In [42]:
temp = pd.read_csv('city_temp_prec.csv', index_col=0)
temp.head()

,year,month,tmax,tmin,prcp,ID,name
0,2017,1,22.225806,2.258065,0.040968,USW00024021,Lander
1,2017,2,38.964286,17.142857,0.039643,USW00024021,Lander
2,2017,3,54.032258,29.548387,0.150000,USW00024021,Lander
3,2017,4,53.900000,30.966667,0.108667,USW00024021,Lander
4,2017,5,65.096774,39.677419,0.070968,USW00024021,Lander


In [43]:
temp = temp.replace({'name':{'EUGENE': 'Eugene'}})
temp['name'] = temp['name'].apply(lambda x: re.sub(r"(\w)([A-Z])", r"\1 \2", x))
temp = temp.groupby(['name','month']).mean().reset_index()

temp['name_split'] = temp['name'].str.split(' ')
temp['state_name'] = temp['name_split'].apply(lambda x: x[-1] if (x[-1].isupper() and len(x[-1])==2) else None)
temp['city_name'] = temp['name_split'].apply(lambda x: ' '.join(x[:-2]) if (x[-1].isupper() and len(x[-1])==2) else ' '.join(x))
temp

,name,month,year,tmax,tmin,prcp,name_split,state_name,city_name
0,Aberdeen,1,2019.0,25.180000,6.267742,0.016194,[Aberdeen],None,Aberdeen
1,Aberdeen,2,2019.0,24.024507,2.581773,0.017259,[Aberdeen],None,Aberdeen
2,Aberdeen,3,2019.0,41.161935,20.584516,0.028581,[Aberdeen],None,Aberdeen
3,Aberdeen,4,2019.0,55.457333,29.265333,0.048533,[Aberdeen],None,Aberdeen
4,Aberdeen,5,2019.0,70.983226,44.136774,0.062065,[Aberdeen],None,Aberdeen
...,...,...,...,...,...,...,...,...,...
2515,Zaneville,8,2019.0,83.398065,62.996129,0.126774,[Zaneville],None,Zaneville
2516,Zaneville,9,2019.0,78.850000,56.886000,0.092467,[Zaneville],None,Zaneville
2517,Zaneville,10,2019.0,67.499785,46.855140,0.113112,[Zaneville],None,Zaneville
2518,Zaneville,11,2019.0,51.367333,32.136667,0.102267,[Zaneville],None,Zaneville


In [46]:
temp

,name,month,year,tmax,tmin,prcp,name_split,state_name,city_name
0,Aberdeen,1,2019.0,25.180000,6.267742,0.016194,[Aberdeen],None,Aberdeen
1,Aberdeen,2,2019.0,24.024507,2.581773,0.017259,[Aberdeen],None,Aberdeen
2,Aberdeen,3,2019.0,41.161935,20.584516,0.028581,[Aberdeen],None,Aberdeen
3,Aberdeen,4,2019.0,55.457333,29.265333,0.048533,[Aberdeen],None,Aberdeen
4,Aberdeen,5,2019.0,70.983226,44.136774,0.062065,[Aberdeen],None,Aberdeen
...,...,...,...,...,...,...,...,...,...
2515,Zaneville,8,2019.0,83.398065,62.996129,0.126774,[Zaneville],None,Zaneville
2516,Zaneville,9,2019.0,78.850000,56.886000,0.092467,[Zaneville],None,Zaneville
2517,Zaneville,10,2019.0,67.499785,46.855140,0.113112,[Zaneville],None,Zaneville
2518,Zaneville,11,2019.0,51.367333,32.136667,0.102267,[Zaneville],None,Zaneville


In [48]:
station_id = pd.read_csv('city_temp_prec.csv', index_col=0).drop(columns=['year', 'month', 'tmax', 'tmin', 'prcp'])
station_id 
station_id = station_id.replace({'name':{'EUGENE': 'Eugene'}})
station_id['name'] = station_id['name'].apply(lambda x: re.sub(r"(\w)([A-Z])", r"\1 \2", x))

station_id = station_id.drop_duplicates()
station_id

,ID,name
0,USW00024021,Lander
0,USW00024018,Cheyenne
0,USW00014897,Wausau
0,USW00094973,Hayward
0,USW00014991,Eau Claire
...,...,...
0,USW00013896,Muscle Shoals
0,USW00013895,Montgomery
0,USW00013838,Mobile
0,USW00003856,Huntsville


In [49]:
temp = temp.merge(station_id, how='left', left_on = 'city_name', right_on = 'name')
temp

,name_x,month,year,tmax,tmin,prcp,name_split,state_name,city_name,ID,name_y
0,Aberdeen,1,2019.0,25.180000,6.267742,0.016194,[Aberdeen],None,Aberdeen,USW00014929,Aberdeen
1,Aberdeen,2,2019.0,24.024507,2.581773,0.017259,[Aberdeen],None,Aberdeen,USW00014929,Aberdeen
2,Aberdeen,3,2019.0,41.161935,20.584516,0.028581,[Aberdeen],None,Aberdeen,USW00014929,Aberdeen
3,Aberdeen,4,2019.0,55.457333,29.265333,0.048533,[Aberdeen],None,Aberdeen,USW00014929,Aberdeen
4,Aberdeen,5,2019.0,70.983226,44.136774,0.062065,[Aberdeen],None,Aberdeen,USW00014929,Aberdeen
...,...,...,...,...,...,...,...,...,...,...,...
2515,Zaneville,8,2019.0,83.398065,62.996129,0.126774,[Zaneville],None,Zaneville,USW00093824,Zaneville
2516,Zaneville,9,2019.0,78.850000,56.886000,0.092467,[Zaneville],None,Zaneville,USW00093824,Zaneville
2517,Zaneville,10,2019.0,67.499785,46.855140,0.113112,[Zaneville],None,Zaneville,USW00093824,Zaneville
2518,Zaneville,11,2019.0,51.367333,32.136667,0.102267,[Zaneville],None,Zaneville,USW00093824,Zaneville


In [53]:
# temp = temp.drop(columns=['state_name'])# 'name_y', 'year', 'name_split', 'name_x'
# temp.to_csv('temp_city_stationID.csv')
temp

,month,tmax,tmin,prcp,city_name,ID
0,1,25.180000,6.267742,0.016194,Aberdeen,USW00014929
1,2,24.024507,2.581773,0.017259,Aberdeen,USW00014929
2,3,41.161935,20.584516,0.028581,Aberdeen,USW00014929
3,4,55.457333,29.265333,0.048533,Aberdeen,USW00014929
4,5,70.983226,44.136774,0.062065,Aberdeen,USW00014929
...,...,...,...,...,...,...
2515,8,83.398065,62.996129,0.126774,Zaneville,USW00093824
2516,9,78.850000,56.886000,0.092467,Zaneville,USW00093824
2517,10,67.499785,46.855140,0.113112,Zaneville,USW00093824
2518,11,51.367333,32.136667,0.102267,Zaneville,USW00093824


In [54]:
# zone['city'] = zone.city.str.title()
zone

,zip_code,zone,city,state,latitude,longitude
0,00705,12b,AIBONITO,PR,18.14,-66.26
1,00610,12b,ANASCO,PR,18.28,-67.14
2,00612,12b,ARECIBO,PR,18.45,-66.73
3,00601,11b,ADJUNTAS,PR,18.16,-66.72
4,00602,12b,AGUADA,PR,18.38,-67.18
...,...,...,...,...,...,...
29749,30293,8a,WOODBURY,GA,32.98,-84.58
29750,30188,7b,WOODSTOCK,GA,34.10,-84.51
29751,30189,7b,WOODSTOCK,GA,34.10,-84.51
29752,30582,7a,YOUNG HARRIS,GA,34.93,-83.84


In [7]:
# !ls

city_temp_prec.csv     state_ab.csv           temp_mon_avg.csv
city_temp_precip.ipynb station_state.csv      temp_prec


In [18]:
# df = pd.read_csv('station_state.csv')
# df

,CITY,STATE,ID,START_DATE
0,Birmingham,Alabama,USW00013876,193001
1,Huntsville,Alabama,USW00003856,195809
2,Mobile,Alabama,USW00013894,194801
3,Montgomery,Alabama,USW00013895,194801
4,Anchorage,Alaska,USW00026451,195204
...,...,...,...,...
210,Wausau,Wisconsin,USW00014897,194112
211,Casper,Wyoming,USW00024089,194808
212,Cheyenne,Wyoming,USH00481675,189501
213,Lander,Wyoming,USW00024021,194801


In [55]:
# state = pd.read_csv('state_ab.csv')
# state = state.drop(columns=['Abbrev'])
# state

In [19]:
# df = df.merge(state, how='left', left_on = ['STATE'], right_on = ['State'])

,CITY,STATE,ID,START_DATE,State,Code
0,Birmingham,Alabama,USW00013876,193001,Alabama,AL
1,Huntsville,Alabama,USW00003856,195809,Alabama,AL
2,Mobile,Alabama,USW00013894,194801,Alabama,AL
3,Montgomery,Alabama,USW00013895,194801,Alabama,AL
4,Anchorage,Alaska,USW00026451,195204,Alaska,AK
...,...,...,...,...,...,...
210,Wausau,Wisconsin,USW00014897,194112,Wisconsin,WI
211,Casper,Wyoming,USW00024089,194808,Wyoming,WY
212,Cheyenne,Wyoming,USH00481675,189501,Wyoming,WY
213,Lander,Wyoming,USW00024021,194801,Wyoming,WY


In [23]:
# df = df.drop(columns=['State', 'START_DATE', 'STATE'])
df_station = df

In [67]:
df_station.to_csv('city_state_code.csv')